In [66]:
import common.common_machine_learning as common
import pandas as pd
import numpy as np
import common.feature_num as feature_num

In [41]:
train = common.cargar_set_optimizado('sets_de_datos/train.csv')

In [42]:
X_test = common.cargar_set_optimizado('sets_de_datos/test.csv')

In [43]:
train['año'] = train['fecha'].dt.year
train['mes'] = train['fecha'].dt.month
train['dia'] = train['fecha'].dt.day

In [44]:
train['metroscubiertos'] = train['metroscubiertos'].fillna(train['metrostotales'])
train['metrostotales'] = train['metrostotales'].fillna(train['metroscubiertos'])

In [45]:
def metros_no_cubiertos(m_totales, m_cubiertos):
    if m_cubiertos >= m_totales:
        return 0
    return m_totales - m_cubiertos
    

train['metros_no_cubiertos'] =\
    train.apply(lambda x: metros_no_cubiertos(x['metrostotales'], x['metroscubiertos']), axis = 1)

In [46]:
train['metros_no_cubiertos'].mean()

14.069029166666667

In [47]:
train['habitaciones'] = train['habitaciones'].fillna(1)
train['garages'] = train['garages'].fillna(0)
train['banos'] = train['banos'].fillna(0)

def cantidad_espacios(df):
    cantidad_espacios = 0
    for col in ['habitaciones', 'garages', 'banos', 'gimnasio', 'usosmultiples', 'piscina']:
        cantidad_espacios += int(df[col])
    return cantidad_espacios

train['cantidad_espacios'] = train.apply(cantidad_espacios, axis = 1)

In [48]:
def metros_x_espacio(cantidad_espacios, metros_cubiertos):
    if cantidad_espacios <= 0 or metros_cubiertos <= 0:
        return 0
    return metros_cubiertos / cantidad_espacios

train['metros_x_espacio'] = train.apply(lambda x: metros_x_espacio(x['cantidad_espacios'], x['metroscubiertos']), axis = 1)

In [49]:
def metros_x_espacios(repeticiones_espacio, metros_x_espacio):
    if repeticiones_espacio <= 0 or metros_x_espacio <= 0:
        return 0
    return repeticiones_espacio * metros_x_espacio

for col in ['habitaciones', 'garages', 'banos', 'gimnasio', 'usosmultiples', 'piscina']:
    train['metros_x_' + col] =\
        train.apply(lambda x: metros_x_espacios(x[col], x['metros_x_espacio']), axis = 1)

In [50]:
train['cosas_cercanas'] = train['escuelascercanas'].astype(int)  + train['centroscomercialescercanos'].astype(int)

In [51]:
train['antiguedad'] = train['antiguedad'].fillna(0)

In [52]:
idzona_mode_x_ciudad = train.groupby(['ciudad'])\
                        .apply(lambda x: x['idzona'].dropna().mode())\
                        .reset_index()\
                        .drop(['level_1'], axis = 1)\
                        .rename(columns = {0 : 'idzona_mode'})

In [53]:
train = train.merge(idzona_mode_x_ciudad, on = 'ciudad', how = 'left')

In [54]:
train['idzona'] = train['idzona'].fillna(train['idzona_mode'])

In [55]:
train = train.drop(['idzona_mode'], axis = 1)

In [56]:
idzona_mode_x_provincia = train.groupby(['provincia'])\
                        .apply(lambda x: x['idzona'].dropna().mode())\
                        .reset_index()\
                        .drop(['level_1'], axis = 1)\
                        .rename(columns = {0 : 'idzona_mode'})

In [57]:
train = train.merge(idzona_mode_x_provincia, on = 'provincia', how = 'left')

In [58]:
train['idzona'] = train['idzona'].fillna(train['idzona_mode'])

In [59]:
train = train.drop(['idzona_mode'], axis = 1)

In [60]:
train['idzona'].isnull().sum()

155

In [61]:
id_zona_mode = train['idzona'].mode()

In [62]:
train.fillna(value = {'idzona' : id_zona_mode.get(0)}, inplace = True)

In [67]:
train = feature_num.completar_lat_lng_con_provincias_y_ciudades(train)

In [70]:
train['provincia'].unique()

array(['Distrito Federal', 'Jalisco', 'Edo. de México', 'Oaxaca',
       'Quintana Roo', 'Colima', 'San luis Potosí', 'Nuevo León',
       'Querétaro', 'Tamaulipas', 'Puebla', 'Yucatán', 'Morelos',
       'Guerrero', 'Tabasco', 'Guanajuato', 'Hidalgo', 'Veracruz',
       'Chihuahua', 'Aguascalientes', 'Sonora', 'Michoacán',
       'Baja California Norte', 'Baja California Sur', 'Coahuila',
       'Durango', 'Sinaloa', 'Chiapas', 'Nayarit', 'Tlaxcala', 'Campeche',
       'Zacatecas', ''], dtype=object)